In [1]:
import requests
import json
import time
from typing import Dict, Any, List

# Server configuration
SERVER_URL = "http://10.52.2.163:8000"  # Your server IP from startup script
API_BASE = f"{SERVER_URL}/api/v1"

# Helper function to submit measurement
def submit_measurement(code: str, 
                      function_name: str,
                      test_cases: List[Dict],
                      trials: int = 5) -> str:
    """Submit energy measurement and return task_id"""
    
    payload = {
        "candidate_code": code,
        "function_name": function_name,
        "test_cases": test_cases,
        "energy_measurement_trials": trials,
        "timeout_seconds": 30
    }
    
    response = requests.post(f"{API_BASE}/measure-socket0", json=payload)
    response.raise_for_status()
    
    result = response.json()
    task_id = result["task_id"]
    
    print(f"✓ Task submitted: {task_id}")
    return task_id

# Helper function to poll results
def get_results(task_id: str, max_wait: int = 60) -> Dict[str, Any]:
    """Poll for measurement results"""
    
    start_time = time.time()
    
    while time.time() - start_time < max_wait:
        response = requests.get(f"{API_BASE}/tasks/{task_id}")
        response.raise_for_status()
        
        result = response.json()
        status = result.get("status")
        
        if status == "completed":
            return result
        elif status == "failed":
            raise RuntimeError(f"Measurement failed: {result.get('error_message')}")
        elif status in ["queued", "running"]:
            print(f"  Status: {status}...", end="\r")
            time.sleep(2)
        else:
            print(f"  Unknown status: {status}")
            time.sleep(2)
    
    raise TimeoutError(f"Measurement timed out after {max_wait}s")

print("✓ Setup complete")
print(f"✓ Server: {SERVER_URL}")

✓ Setup complete
✓ Server: http://10.52.2.163:8000


In [2]:
print("=" * 70)
print("FIBONACCI ENERGY TEST - Efficient vs Inefficient")
print("=" * 70)

# ============================================================================
# 1. Define Solutions
# ============================================================================

# EFFICIENT: Dynamic Programming O(n)
efficient_fib = '''
def fibonacci(n):
    """Efficient: Dynamic programming - O(n)"""
    if n <= 1:
        return n
    a, b = 0, 1
    for _ in range(2, n + 1):
        a, b = b, a + b
    return b
'''

# INEFFICIENT: Naive Recursion O(2^n)
inefficient_fib = '''
def fibonacci(n):
    """Inefficient: Naive recursion - O(2^n)"""
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)
'''

print("\n✓ Efficient:   O(n) dynamic programming")
print("✓ Inefficient: O(2^n) naive recursion")

# ============================================================================
# 2. Generate Test Cases
# ============================================================================

# Use fibonacci numbers that show dramatic difference
# For O(2^n), even n=30-35 will be MUCH slower than O(n)
fib_test_cases = [
    {"test_id": "fib_30", "inputs": [30], "expected_output": 832040},
    {"test_id": "fib_32", "inputs": [32], "expected_output": 2178309},
    {"test_id": "fib_35", "inputs": [35], "expected_output": 9227465},
]

print(f"\n✓ Test cases: Fibonacci(30, 32, 35)")

# ============================================================================
# 3. Submit Measurements
# ============================================================================

print("\n[1/2] Submitting EFFICIENT Fibonacci (DP)...")
efficient_fib_task = submit_measurement(
    code=efficient_fib,
    function_name="fibonacci",
    test_cases=fib_test_cases,
    trials=5
)

time.sleep(1)

print("\n[2/2] Submitting INEFFICIENT Fibonacci (Recursion)...")
inefficient_fib_task = submit_measurement(
    code=inefficient_fib,
    function_name="fibonacci",
    test_cases=fib_test_cases,
    trials=5
)

# ============================================================================
# 4. Wait for Results
# ============================================================================

print("\n[1/2] Waiting for EFFICIENT results...")
eff_fib_result = get_results(efficient_fib_task, max_wait=180)

print("\n[2/2] Waiting for INEFFICIENT results...")
ineff_fib_result = get_results(inefficient_fib_task, max_wait=180)

# ============================================================================
# 5. Compare Results
# ============================================================================

print("\n" + "=" * 70)
print("FIBONACCI ENERGY MEASUREMENT RESULTS")
print("=" * 70)

eff_fib = eff_fib_result["energy_metrics"]
ineff_fib = ineff_fib_result["energy_metrics"]

print(f"\n{'Metric':<35} {'Efficient O(n)':<18} {'Inefficient O(2^n)':<18} {'Ratio':<10}")
print("-" * 70)

metrics = [
    ("Total Energy (J)", "median_total_energy_joules"),
    ("Package Energy (J)", "median_package_energy_joules"),
    ("DRAM Energy (J)", "median_ram_energy_joules"),
    ("Execution Time (s)", "median_execution_time_seconds"),
    #("Power (W)", "power_consumption_watts"),
    ("Energy per Test (J)", "energy_per_test_case_joules"),
]

for label, key in metrics:
    eff_val = eff_fib[key]
    ineff_val = ineff_fib[key]
    ratio = ineff_val / eff_val if eff_val > 0 else 0
    print(f"{label:<35} {eff_val:<18.3f} {ineff_val:<18.3f} {ratio:<10.2f}x")

print("=" * 70)

# Verdict
energy_ratio = ineff_fib["median_total_energy_joules"] / eff_fib["median_total_energy_joules"]
time_ratio = ineff_fib["median_execution_time_seconds"] / eff_fib["median_execution_time_seconds"]

print(f"\n FIBONACCI RESULTS:")
print(f"  Energy Ratio:  Inefficient uses {energy_ratio:.1f}x MORE energy")
print(f"  Time Ratio:    Inefficient takes {time_ratio:.1f}x MORE time")

print(f"\n✓ Energy tracks algorithmic complexity ({energy_ratio:.1f}x)")

print("\n" + "=" * 70)

FIBONACCI ENERGY TEST - Efficient vs Inefficient

✓ Efficient:   O(n) dynamic programming
✓ Inefficient: O(2^n) naive recursion

✓ Test cases: Fibonacci(30, 32, 35)

[1/2] Submitting EFFICIENT Fibonacci (DP)...
✓ Task submitted: be5db691-b038-4eb8-b20b-f09ce831d88e

[2/2] Submitting INEFFICIENT Fibonacci (Recursion)...
✓ Task submitted: e2be5367-a875-4149-97a8-07b781a569e5

[1/2] Waiting for EFFICIENT results...
  Status: running...
[2/2] Waiting for INEFFICIENT results...
  Status: running...
FIBONACCI ENERGY MEASUREMENT RESULTS

Metric                              Efficient O(n)     Inefficient O(2^n) Ratio     
----------------------------------------------------------------------
Total Energy (J)                    4.285              36.431             8.50      x
Package Energy (J)                  2.066              17.393             8.42      x
DRAM Energy (J)                     2.219              19.038             8.58      x
Execution Time (s)                  0.307        

In [3]:
# Problem: Two Sum (LeetCode #1)
# Given array of integers and target, return indices of two numbers that add up to target

# Generate test cases with sufficient size to show energy difference
def generate_test_cases(sizes=[100, 200, 300, 500, 1000]):
    """Generate Two Sum test cases of varying sizes"""
    test_cases = []
    
    for idx, size in enumerate(sizes):
        # Create array [1, 2, 3, ..., size]
        nums = list(range(1, size + 1))
        # Target is last two numbers
        target = nums[-1] + nums[-2]
        # Expected output is indices of last two elements
        expected = [size - 2, size - 1]
        
        test_cases.append({
            "test_id": f"test_{idx+1}_size_{size}",
            "inputs": [nums, target],
            "expected_output": expected
        })
    
    return test_cases

# Generate test cases
test_cases = generate_test_cases()

print(f"✓ Generated {len(test_cases)} test cases")
print(f"✓ Sizes: {[100, 200, 300, 500, 1000]}")
print(f"\nExample test case:")
print(f"  Input size: {len(test_cases[0]['inputs'][0])}")
print(f"  Target: {test_cases[0]['inputs'][1]}")
print(f"  Expected: {test_cases[0]['expected_output']}")

✓ Generated 5 test cases
✓ Sizes: [100, 200, 300, 500, 1000]

Example test case:
  Input size: 100
  Target: 199
  Expected: [98, 99]


In [4]:
# EFFICIENT SOLUTION: O(n) with hash map
efficient_solution = '''
def two_sum(nums, target):
    """Efficient: Single pass with hash map - O(n)"""
    seen = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in seen:
            return [seen[complement], i]
        seen[num] = i
    return []
'''

# INEFFICIENT SOLUTION: O(n²) with nested loops
inefficient_solution = '''
def two_sum(nums, target):
    """Inefficient: Nested loops - O(n²)"""
    n = len(nums)
    for i in range(n):
        for j in range(i + 1, n):
            if nums[i] + nums[j] == target:
                return [i, j]
    return []
'''

print("=" * 60)
print("Solution Comparison")
print("=" * 60)
print("\n1. EFFICIENT (Hash Map):")
print("   - Time Complexity: O(n)")
print("   - Space: O(n)")
print("   - Strategy: Single pass, constant-time lookups")

print("\n2. INEFFICIENT (Nested Loops):")
print("   - Time Complexity: O(n²)")
print("   - Space: O(1)")
print("   - Strategy: Check every pair")

print("\n✓ Both solutions ready for energy measurement")
print("✓ Hypothesis: Inefficient solution should use MORE joules")

Solution Comparison

1. EFFICIENT (Hash Map):
   - Time Complexity: O(n)
   - Space: O(n)
   - Strategy: Single pass, constant-time lookups

2. INEFFICIENT (Nested Loops):
   - Time Complexity: O(n²)
   - Space: O(1)
   - Strategy: Check every pair

✓ Both solutions ready for energy measurement
✓ Hypothesis: Inefficient solution should use MORE joules


In [5]:
print("=" * 70)
print("RETRYING: Using MUCH larger test cases")
print("=" * 70)

# Generate MUCH larger test cases to amplify difference
def generate_large_test_cases():
    """Generate large test cases to show O(n²) vs O(n) difference"""
    test_cases = []
    
    # Sizes that will show dramatic difference
    sizes = [5000, 10000, 15000]
    
    for idx, size in enumerate(sizes):
        nums = list(range(1, size + 1))
        target = nums[-1] + nums[-2]
        expected = [size - 2, size - 1]
        
        test_cases.append({
            "test_id": f"large_test_{idx+1}_size_{size}",
            "inputs": [nums, target],
            "expected_output": expected
        })
    
    return test_cases

# Generate larger test cases
large_test_cases = generate_large_test_cases()

print(f"\n✓ Generated {len(large_test_cases)} LARGE test cases")
print(f"✓ Sizes: [5000, 10000, 15000] elements")
print(f"✓ O(n²) should take MUCH longer than O(n)")

# Submit both solutions with large test cases
print("\n[1/2] Submitting EFFICIENT solution (Hash Map) with LARGE inputs...")
efficient_large_task = submit_measurement(
    code=efficient_solution,
    function_name="two_sum",
    test_cases=large_test_cases,
    trials=3  # Fewer trials since each is longer
)

time.sleep(1)

print("\n[2/2] Submitting INEFFICIENT solution (Nested Loops) with LARGE inputs...")
inefficient_large_task = submit_measurement(
    code=inefficient_solution,
    function_name="two_sum",
    test_cases=large_test_cases,
    trials=3
)

print("\nTask IDs:")
print(f"  Efficient:   {efficient_large_task}")
print(f"  Inefficient: {inefficient_large_task}")
print("\nWaiting for results (this may take 30-60s)...")

RETRYING: Using MUCH larger test cases

✓ Generated 3 LARGE test cases
✓ Sizes: [5000, 10000, 15000] elements
✓ O(n²) should take MUCH longer than O(n)

[1/2] Submitting EFFICIENT solution (Hash Map) with LARGE inputs...
✓ Task submitted: e4792587-67ea-4833-bde4-a8a9fc0c1d9a

[2/2] Submitting INEFFICIENT solution (Nested Loops) with LARGE inputs...
✓ Task submitted: 1eb60778-6823-4fb3-a7e6-d0ba9fe6e41b

Task IDs:
  Efficient:   e4792587-67ea-4833-bde4-a8a9fc0c1d9a
  Inefficient: 1eb60778-6823-4fb3-a7e6-d0ba9fe6e41b

Waiting for results (this may take 30-60s)...


In [6]:
metrics_to_compare = [
    ("Total Energy (J)", "median_total_energy_joules"),
    ("Package Energy (J)", "median_package_energy_joules"),
    ("DRAM Energy (J)", "median_ram_energy_joules"),
    ("Execution Time (s)", "median_execution_time_seconds"),
    ("Energy per Test Case (J)", "energy_per_test_case_joules"),
]

In [7]:
print("\n[1/2] Waiting for EFFICIENT solution (large inputs)...")
efficient_large_result = get_results(efficient_large_task, max_wait=180)

print("\n[2/2] Waiting for INEFFICIENT solution (large inputs)...")
inefficient_large_result = get_results(inefficient_large_task, max_wait=180)

print("\n" + "=" * 70)
print("ENERGY MEASUREMENT RESULTS - LARGE TEST CASES")
print("=" * 70)

# Extract metrics
eff_large = efficient_large_result["energy_metrics"]
ineff_large = inefficient_large_result["energy_metrics"]

# Display comparison
print(f"\n{'Metric':<35} {'Efficient':<15} {'Inefficient':<15} {'Ratio':<10}")
print("-" * 70)

for label, key in metrics_to_compare:
    eff_val = eff_large[key]
    ineff_val = ineff_large[key]
    ratio = ineff_val / eff_val if eff_val > 0 else 0
    
    print(f"{label:<35} {eff_val:<15.3f} {ineff_val:<15.3f} {ratio:<10.2f}")

print("=" * 70)

# Detailed verdict
total_ratio = ineff_large["median_total_energy_joules"] / eff_large["median_total_energy_joules"]
time_ratio = ineff_large["median_execution_time_seconds"] / eff_large["median_execution_time_seconds"]

print(f"\n✓ RESULT:")
print(f"  Energy Ratio:    Inefficient uses {total_ratio:.2f}x MORE energy")
print(f"  Time Ratio:      Inefficient takes {time_ratio:.2f}x MORE time")
print(f"  Energy/Time:     {total_ratio/time_ratio:.2f}x (should be ~1.0)")

if total_ratio > 2.0:
    print(f"\n HYPOTHESIS CONFIRMED!")
    print(f"   Energy consumption tracks algorithmic complexity!")
    print(f"   O(n²) solution uses {total_ratio:.1f}x more energy than O(n)")
else:
    print(f"\n⚠ Ratio still small ({total_ratio:.2f}x)")
    print(f"   May need even larger test cases or different algorithm")


[1/2] Waiting for EFFICIENT solution (large inputs)...
  Status: running...
[2/2] Waiting for INEFFICIENT solution (large inputs)...
  Status: running...
ENERGY MEASUREMENT RESULTS - LARGE TEST CASES

Metric                              Efficient       Inefficient     Ratio     
----------------------------------------------------------------------
Total Energy (J)                    5.215           147.322         28.25     
Package Energy (J)                  2.512           70.598          28.10     
DRAM Energy (J)                     2.703           76.725          28.39     
Execution Time (s)                  0.372           10.690          28.71     
Energy per Test Case (J)            1.738           49.107          28.25     

✓ RESULT:
  Energy Ratio:    Inefficient uses 28.25x MORE energy
  Time Ratio:      Inefficient takes 28.71x MORE time
  Energy/Time:     0.98x (should be ~1.0)

 HYPOTHESIS CONFIRMED!
   Energy consumption tracks algorithmic complexity!
   O(n²) solut